# Recession-Proof Stock Screener

Find low-value stocks with positive long-term trends in recession-resistant sectors.

**22 Categories covering 150+ stocks:**
- **Utilities:** Electric, Water
- **Consumer Defensive:** Staples, Food & Beverage, Household Products, Grocery
- **Retail:** Discount Retail
- **Healthcare:** General, Pharmaceuticals, Health Insurance
- **Telecom:** Communication Services
- **REITs:** Healthcare, Residential, Essential/Triple-Net
- **Financial:** Insurance, Regional Banks
- **Infrastructure:** Waste Management, Defense/Aerospace, Railroads, Infrastructure
- **Energy:** Midstream/Pipelines
- **Precious Metals:** Gold Miners
- **Income:** Dividend Aristocrats
- **Alternative:** Sin Stocks

In [ ]:
# Install dependencies
!pip install yfinance pandas numpy -q

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Recession-proof sector categories
SECTORS = {
    # Defensive Utilities
    "utilities": ["NEE", "DUK", "SO", "D", "AEP", "XEL", "ES", "WEC", "ED", "EIX"],
    "water_utilities": ["AWK", "WTR", "WTRG", "SJW", "AWR", "CWT", "MSEX", "YORW"],

    # Consumer Defensive
    "consumer_staples": ["PG", "KO", "PEP", "WMT", "COST", "CL", "KMB", "GIS", "K", "HSY"],
    "food_beverage": ["KO", "PEP", "MDLZ", "KHC", "GIS", "CPB", "SJM", "CAG", "HRL", "TSN"],
    "household_products": ["PG", "CL", "CHD", "CLX", "KMB", "SPB", "EPC", "CENT"],

    # Retail
    "discount_retail": ["WMT", "COST", "DG", "DLTR", "TJX", "ROST", "BJ", "OLLI", "FIVE", "PSMT"],
    "grocery": ["KR", "ACI", "SFM", "GO", "NGVC", "WMK", "VLGEA"],

    # Healthcare
    "healthcare": ["JNJ", "UNH", "PFE", "MRK", "ABBV", "LLY", "BMY", "AMGN", "GILD", "CVS"],
    "pharmaceuticals": ["PFE", "MRK", "ABBV", "LLY", "BMY", "AMGN", "GILD", "VTRS", "TAK", "NVO"],
    "health_insurance": ["UNH", "ELV", "CI", "HUM", "CNC", "MOH"],

    # Telecom & Communication
    "telecom": ["T", "VZ", "TMUS", "CMCSA", "CHTR"],

    # Real Estate (Defensive REITs)
    "reits_healthcare": ["WELL", "VTR", "OHI", "PEAK", "HR", "DOC", "SBRA", "LTC"],
    "reits_residential": ["AVB", "EQR", "ESS", "MAA", "UDR", "CPT", "INVH", "AMH"],
    "reits_essential": ["O", "WPC", "NNN", "ADC", "STOR", "FCPT", "EPRT"],

    # Financial (Defensive)
    "insurance": ["BRK-B", "PGR", "ALL", "TRV", "CB", "MET", "PRU", "AFL", "AIG", "HIG"],
    "regional_banks": ["USB", "PNC", "TFC", "FITB", "RF", "KEY", "CFG", "MTB", "HBAN"],

    # Infrastructure & Industrial
    "waste_management": ["WM", "RSG", "WCN", "CWST", "GFL", "CLH", "SRCL"],
    "defense_aerospace": ["LMT", "RTX", "NOC", "GD", "BA", "LHX", "HII", "TXT", "LDOS"],
    "railroads": ["UNP", "CSX", "NSC", "CP", "CNI"],
    "infrastructure": ["AMT", "CCI", "SBAC", "NEE", "AEP", "PCG", "SRE", "WMB", "KMI"],

    # Energy (Defensive - Pipelines/Storage)
    "midstream_energy": ["EPD", "ET", "MPLX", "WMB", "KMI", "OKE", "TRGP", "PAA"],

    # Precious Metals (Recession Hedge)
    "gold_miners": ["NEM", "GOLD", "AEM", "FNV", "WPM", "RGLD", "KGC", "AGI"],

    # Dividend Aristocrats (25+ years of dividend growth)
    "dividend_aristocrats": ["JNJ", "PG", "KO", "PEP", "MMM", "ABT", "ABBV", "MCD",
                            "WMT", "XOM", "CVX", "CL", "EMR", "GPC", "ITW", "SWK"],

    # Sin Stocks (Recession-resistant vices)
    "sin_stocks": ["MO", "PM", "BTI", "STZ", "BF-B", "DEO", "TAP", "SAM", "WYNN", "LVS"],

    # Small/Mid-Cap Defense Contractors (competing for US contracts)
    "defense_smallcap": [
        "KTOS",   # Kratos Defense - drones, unmanned systems, missile defense
        "MRCY",   # Mercury Systems - defense electronics, processing
        "AVAV",   # AeroVironment - tactical drones, loitering munitions
        "BWXT",   # BWX Technologies - nuclear components, reactors
        "PSN",    # Parsons Corp - defense engineering, cybersecurity
        "CACI",   # CACI International - defense IT, intel services
        "SAIC",   # Science Applications - defense IT services
        "BAH",    # Booz Allen Hamilton - defense consulting
        "AJRD",   # Aerojet Rocketdyne - rocket propulsion
        "TDG",    # TransDigm - aerospace components
        "HEI",    # HEICO - aircraft replacement parts
        "CW",     # Curtiss-Wright - defense components
        "MOG-A",  # Moog Inc - flight controls, defense systems
        "PLTR",   # Palantir - defense data analytics, AI
        "RKLB",   # Rocket Lab - small launch vehicles, space
        "RDW",    # Redwire - space infrastructure
        "IRDM",   # Iridium - satellite communications
        "GILT",   # Gilat Satellite - satellite networking
        "VSAT",   # Viasat - defense communications
        "AXON",   # Axon - law enforcement tech
        "SWBI",   # Smith & Wesson - firearms
        "RGR",    # Sturm Ruger - firearms
        "POWW",   # AMMO Inc - ammunition
    ],

    # Space & Hypersonics (emerging defense tech)
    "defense_space": [
        "RKLB",   # Rocket Lab
        "LUNR",   # Intuitive Machines - lunar landers
        "RDW",    # Redwire - space manufacturing
        "MNTS",   # Momentus - space transport
        "BKSY",   # BlackSky - satellite imagery intel
        "PL",     # Planet Labs - earth imaging
        "ASTS",   # AST SpaceMobile - space-based cellular
        "IRDM",   # Iridium
        "SPIR",   # Spire Global - satellite data
    ],
}

print(f"Loaded {len(SECTORS)} categories")
print("Categories:", list(SECTORS.keys()))

In [ ]:
def get_stock_data(symbol: str) -> dict | None:
    """Fetch stock data and key metrics."""
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info

        # Get historical data for trend analysis (2 years)
        hist = ticker.history(period="2y")
        if hist.empty:
            return None

        # Calculate trend metrics
        current_price = hist["Close"].iloc[-1]
        price_1y_ago = hist["Close"].iloc[len(hist)//2] if len(hist) > 250 else hist["Close"].iloc[0]
        price_2y_ago = hist["Close"].iloc[0]

        return {
            "symbol": symbol,
            "name": info.get("shortName", symbol),
            "sector": info.get("sector", "Unknown"),
            "price": current_price,
            "pe_ratio": info.get("trailingPE"),
            "forward_pe": info.get("forwardPE"),
            "pb_ratio": info.get("priceToBook"),
            "dividend_yield": info.get("dividendYield", 0) or 0,
            "market_cap": info.get("marketCap", 0),
            "beta": info.get("beta"),
            "52w_high": info.get("fiftyTwoWeekHigh"),
            "52w_low": info.get("fiftyTwoWeekLow"),
            "1y_return": ((current_price - price_1y_ago) / price_1y_ago) * 100,
            "2y_return": ((current_price - price_2y_ago) / price_2y_ago) * 100,
            "avg_volume": info.get("averageVolume", 0),
        }
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")
        return None


def calculate_value_score(stock: dict) -> float:
    """Calculate a value score (higher = better value)."""
    score = 0

    # P/E ratio scoring (lower is better)
    pe = stock.get("pe_ratio")
    if pe and pe > 0:
        if pe < 15:
            score += 3
        elif pe < 20:
            score += 2
        elif pe < 25:
            score += 1

    # P/B ratio scoring (lower is better)
    pb = stock.get("pb_ratio")
    if pb and pb > 0:
        if pb < 1.5:
            score += 3
        elif pb < 3:
            score += 2
        elif pb < 5:
            score += 1

    # Dividend yield scoring (higher is better for stability)
    div_yield = stock.get("dividend_yield", 0)
    if div_yield > 0.04:
        score += 3
    elif div_yield > 0.02:
        score += 2
    elif div_yield > 0.01:
        score += 1

    # Beta scoring (lower beta = more stable/recession-proof)
    beta = stock.get("beta")
    if beta:
        if beta < 0.8:
            score += 3
        elif beta < 1.0:
            score += 2
        elif beta < 1.2:
            score += 1

    return score


def calculate_trend_score(stock: dict) -> float:
    """Calculate trend score (positive = uptrend)."""
    score = 0

    # 1-year return
    ret_1y = stock.get("1y_return", 0)
    if ret_1y > 20:
        score += 3
    elif ret_1y > 10:
        score += 2
    elif ret_1y > 0:
        score += 1
    elif ret_1y < -20:
        score -= 2

    # 2-year return
    ret_2y = stock.get("2y_return", 0)
    if ret_2y > 30:
        score += 3
    elif ret_2y > 15:
        score += 2
    elif ret_2y > 0:
        score += 1

    return score


def screen_stocks(category: str = None, min_value_score: int = 0, min_trend_score: int = 0) -> pd.DataFrame:
    """
    Screen stocks based on category and scoring criteria.

    Args:
        category: Sector category (utilities, consumer_staples, healthcare, etc.)
                 If None, screens all categories
        min_value_score: Minimum value score to include (default 0)
        min_trend_score: Minimum trend score to include (default 0)

    Returns:
        DataFrame of qualifying stocks sorted by combined score
    """
    if category:
        symbols = SECTORS.get(category, [])
        if not symbols:
            print(f"Unknown category: {category}")
            print(f"Available categories: {list(SECTORS.keys())}")
            return pd.DataFrame()
    else:
        # Get all unique symbols from all categories
        symbols = list(set(sym for syms in SECTORS.values() for sym in syms))

    print(f"Screening {len(symbols)} stocks...")

    results = []
    for i, symbol in enumerate(symbols):
        print(f"  [{i+1}/{len(symbols)}] Analyzing {symbol}...", end="\r")
        data = get_stock_data(symbol)
        if data:
            data["value_score"] = calculate_value_score(data)
            data["trend_score"] = calculate_trend_score(data)
            data["combined_score"] = data["value_score"] + data["trend_score"]
            results.append(data)

    print(" " * 50)  # Clear progress line

    if not results:
        return pd.DataFrame()

    df = pd.DataFrame(results)

    # Filter by minimum scores
    df = df[(df["value_score"] >= min_value_score) & (df["trend_score"] >= min_trend_score)]

    # Sort by combined score (descending)
    df = df.sort_values("combined_score", ascending=False)

    return df

print("Functions loaded!")

## Screen All Categories
Run this cell to screen all recession-proof stocks:

In [ ]:
# Screen ALL categories
results = screen_stocks(category=None)

# Display top results
display_cols = ["symbol", "name", "price", "pe_ratio", "pb_ratio", 
                "dividend_yield", "beta", "1y_return", "value_score", 
                "trend_score", "combined_score"]

results[display_cols].head(20)

## Screen by Category
Change the category variable to screen specific sectors:

In [ ]:
# Choose a category from the list above
# Examples: utilities, water_utilities, consumer_staples, food_beverage,
#           household_products, discount_retail, grocery, healthcare,
#           pharmaceuticals, health_insurance, telecom, reits_healthcare,
#           reits_residential, reits_essential, insurance, regional_banks,
#           waste_management, defense_aerospace, railroads, infrastructure,
#           midstream_energy, gold_miners, dividend_aristocrats, sin_stocks

category = "dividend_aristocrats"  # <-- CHANGE THIS

results = screen_stocks(category=category)
results[["symbol", "name", "price", "pe_ratio", "dividend_yield", 
         "beta", "1y_return", "combined_score"]]

## Filter for Best Value Stocks
Find stocks with high value AND trend scores:

In [ ]:
# Screen with minimum score requirements
results = screen_stocks(
    category=None,        # All categories
    min_value_score=5,    # Must have good value metrics
    min_trend_score=2     # Must have positive trend
)

print(f"\nFound {len(results)} stocks matching criteria:\n")
results[["symbol", "name", "price", "pe_ratio", "pb_ratio",
         "dividend_yield", "beta", "1y_return", "combined_score"]]

## Visualize Results

In [ ]:
import matplotlib.pyplot as plt

# Get all stocks
all_results = screen_stocks(category=None)

# Plot Value Score vs Trend Score
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
ax1 = axes[0]
scatter = ax1.scatter(all_results["value_score"], all_results["trend_score"], 
                      c=all_results["combined_score"], cmap="RdYlGn", s=100, alpha=0.7)
for i, row in all_results.head(10).iterrows():
    ax1.annotate(row["symbol"], (row["value_score"], row["trend_score"]), fontsize=8)
ax1.set_xlabel("Value Score")
ax1.set_ylabel("Trend Score")
ax1.set_title("Value vs Trend Score")
plt.colorbar(scatter, ax=ax1, label="Combined Score")

# Top 10 by combined score
ax2 = axes[1]
top10 = all_results.head(10)
bars = ax2.barh(top10["symbol"], top10["combined_score"], color="steelblue")
ax2.set_xlabel("Combined Score")
ax2.set_title("Top 10 Recession-Proof Value Stocks")
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

## Add Custom Stocks
Analyze your own stock symbols:

In [ ]:
# Add your own symbols to analyze
custom_symbols = ["AAPL", "MSFT", "BRK-B", "JPM", "V"]  # <-- ADD YOUR SYMBOLS

custom_results = []
for symbol in custom_symbols:
    print(f"Analyzing {symbol}...")
    data = get_stock_data(symbol)
    if data:
        data["value_score"] = calculate_value_score(data)
        data["trend_score"] = calculate_trend_score(data)
        data["combined_score"] = data["value_score"] + data["trend_score"]
        custom_results.append(data)

custom_df = pd.DataFrame(custom_results).sort_values("combined_score", ascending=False)
custom_df[["symbol", "name", "price", "pe_ratio", "dividend_yield", 
           "beta", "1y_return", "combined_score"]]